# 1. Data prep

## 1.1 Download data

In [1]:
!pip install gdown
!gdown 14Rq-YANUv8qyi4Ze8ReEAEu_uxgcV_Yk
!unzip /kaggle/working/ViMMRC.zip 

Downloading...
From: https://drive.google.com/uc?id=14Rq-YANUv8qyi4Ze8ReEAEu_uxgcV_Yk
To: /kaggle/working/ViMMRC.zip
100%|████████████████████████████████████████| 770k/770k [00:00<00:00, 80.6MB/s]
Archive:  /kaggle/working/ViMMRC.zip
   creating: ViMMRC/
  inflating: ViMMRC/.DS_Store        
   creating: __MACOSX/
   creating: __MACOSX/ViMMRC/
  inflating: __MACOSX/ViMMRC/._.DS_Store  
   creating: ViMMRC/test/
  inflating: ViMMRC/test/grade_3_9_test.json  
   creating: __MACOSX/ViMMRC/test/
  inflating: __MACOSX/ViMMRC/test/._grade_3_9_test.json  
  inflating: ViMMRC/test/grade_4_1_test.json  
  inflating: __MACOSX/ViMMRC/test/._grade_4_1_test.json  
  inflating: ViMMRC/test/grade_2_0_test.json  
  inflating: __MACOSX/ViMMRC/test/._grade_2_0_test.json  
  inflating: ViMMRC/test/grade_5_8_test.json  
  inflating: __MACOSX/ViMMRC/test/._grade_5_8_test.json  
  inflating: ViMMRC/test/grade_5_19_test.json  
  inflating: __MACOSX/ViMMRC/test/._grade_5_19_test.json  
  inflating: ViMMRC/te

## 1.2 Exploratory analysis

In [2]:
from datasets import load_dataset, load_metric
dataset = load_dataset("/kaggle/working/ViMMRC")

Resolving data files:   0%|          | 0/292 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/83 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/42 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/ViMMRC-00b2f24bc0bca80a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'questions', 'options', 'answers'],
        num_rows: 292
    })
    test: Dataset({
        features: ['article', 'questions', 'options', 'answers'],
        num_rows: 83
    })
    validation: Dataset({
        features: ['article', 'questions', 'options', 'answers'],
        num_rows: 42
    })
})

In [4]:
dataset['train'][0]

{'article': 'Ôi lạ quá! Bỗng nhiên bé nhìn thấy mình biến thành cô bé tí hon. Những đóa hoa đang nở để đón mùa xuân. Còn bướm ong đang bay rập rờn. Nắng vàng làm những hạt sương lóng lánh. Thế mà bé cứ tưởng là bong bóng. Bé sờ vào, nó vỡ ra tạo thành một dòng nước cuốn bé trôi xuống đất. Bé sợ quá, thức dậy. Bé nói: “Thì ra mình nằm mơ.”.',
 'questions': ['Bài đọc miêu tả mùa nào trong năm?',
  'Bướm ong đang bay được miêu tả như thế nào?',
  'Bé tưởng cái gì là bong bóng ?',
  'Câu chuyện trên được mô tả khi bé trong trạng thái nào?',
  'Thứ gì đã tạo thành dòng nước và cuốn bé trôi xuống đất?'],
 'options': [['Mùa xuân.', 'Mùa thu.', 'Mùa hè.', 'Mùa đông.'],
  ['Tung tăng.', 'Rập rờn.', 'Vù vù.', 'Như bong bóng.'],
  ['Giọt nước mưa đọng trên lá.',
   'Những hạt sương lóng lánh.',
   'Chùm quả đẫm nước mưa sau cơn mưa.',
   'Những con ong bướm mùa xuân.'],
  ['Trong giấc mơ.', 'Giấc ngủ sâu.', 'Đang thức.', 'Đang đi chơi.'],
  ['Hạt sương.', 'Bong bóng.', 'Nắng vàng.', 'Hoa.']],
 'a

In [6]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [7]:
show_random_elements(dataset["train"])

,article,questions,options,answers
0,"Đà Lạt, một buổi chiều cuối năm, mưa giông vừa tạnh. Mặt trời hé nắng vàng vàng, không khí nhẹ và trong, mát rười rượi, kích thích đến tim óc.\n\nTôi mở cửa ra bao lơn nhìn sang rừng thông. Tôi để ý nhìn những cây thông cao, không nứt nẻ, cành sần sùi, cong queo một cách mĩ thuật không ngờ. Dưới chân cây, cỏ dại mọc dày, cao, lá thon, dọc ngang, lá đan dày um tùm.\n\nTôi nhìn ra xa hơn. Hồ Đà Lạt lặng im, mặt nước xanh phản chiếu đồi núi và rừng thông. Màu xanh và sự im lặng, cảnh bao la của núi rừng và không khí mát mẻ như làm châm vào da, tất cả nhè nhẹ đưa óc tôi liên tưởng đền phong cảnh sứ Phần Lan tôi đã nhiều lần tưởng tượng qua sách vở.\n\nTôi đang mơ màng tưởng tượng thì chợt vang lên tiếng chim hoàng anh hót. Tôi đưa mắt nhìn xem chim đậu ở đâu mà hót. Mỗi lần nghe tiếng chim hoàng anh là tôi phải nghĩ đến những cây bàng xanh mướt, vòm trời xanh có mây nhẹ như bông. Đà Lạt có chim hoàng anh hót là điều tôi không bao giờ ngờ. Nhưng kìa, trên một cây thông gãy có một chú hoàng anh nhỏ đang mổ vỏ thông tìm mồi. Mình chim thon thon, lông mượt màu vàng nghệ, hoà hợp với giọng hót ấm áp. Óc tôi đột nhiên thấy êm ái vô cùng.","[Nên chọn tên nào cho bài văn?, Vật nào không được tác giả miêu tả trong bài?, Thời tiết ở Đà Lạt như thế nào?, Nghe tiếng hoàng anh hót, tác giả liên tưởng đến điều gì?, Không gian của Đà Lạt có đặc điểm gì?]","[[Một buổi sáng Đà Lạt., Một buổi chiều Đà Lạt., Những âm thanh ở Đà Lạt., Những hình ảnh Đà Lạt.], [Núi., Tiếng chim., Cây thông., Suối.], [Nóng ẩm., Mát mẻ., Lạnh và khô., Nóng.], [Màu nắng của những ngày đẹp trời., Rừng thông xanh và và mặt hồ màu ngọc bích., Những cây bàng xanh mướt, vòm trời xanh có mây nhẹ như bông., Những cây thông xanh trong những ngày đẹp trời.], [Sôi động và náo nhiệt., Lắng đọng và trầm buồn., Yên tĩnh và thơ mộng., Sôi động và thơ mộng.]]","[B, D, B, C, C]"
1,"Tôi được tặng một chiếc xe đạp rất đẹp nhân dịp sinh nhật của mình. Trong một lần đạp xe ra công viên dạo chơi, có một cậu bé cứ quẩn quanh ngắm nhìn chiếc xe với vẻ thích thú và ngưỡng mộ.\n\n- Chiếc xe này của bạn đấy à? - Cậu bé hỏi.\n\n- Anh trai mình đã tặng nhân dịp sinh nhật của mình đấy.\n\n- Tôi trả lời, không giấu vẻ tự hào và mãn nguyện.\n\n- Ồ ước gì tớ ... - Cậu bé ngập ngừng.\n\nDĩ nhiên là tôi biết cậu bé đang ước điều gì rồi, cậu ấy hẳn đang ước ao có được một người anh như thế. Nhưng câu nói của cậu thật bất ngờ đối với tôi.\n\n- Ước gì tớ có thể trở thành một người anh như thế!\n\n- Cậu ấy nói chậm rãi và gương mặt lộ rõ vẻ quyết tâm. Sau đó cậu đi về phía chiếc ghế đá sau lưng tôi, nơi đứa em trai nhỏ tàn tật của cậu đang ngồi và nói: ""Đến sinh nhật nào đó của em, anh sẽ mua tặng em chiếc xe lăn, em nhé!""","[Nhân vật ""tôi"" trong câu chuyện có chuyện gì vui?, Nhân vật ""tôi"" đoán cậu bé ước mơ điều gì?, Tình tiết nào trong câu chuyện làm em bất ngờ, cảm động nhất?, Tìm các đại từ xưng hô trong các câu hội thoại của bài văn trên?, Nêu cảm nghĩ của em về nhân vật cậu bé trong câu chuyện ""Một người anh như thế"", Cậu bé ước mình có thể trở thành ""một người anh như thế"" nghĩa là ước điều gì?, Điều gì khiến nhân vật ""tôi"" tự hào và mãn nguyện?]","[[Được đi chơi công viên., Sắp được món quà sinh nhật., Được tặng một chiếc xe đạp rất đẹp nhân dịp ngày sinh nhật., Tất cả đều sai.], [Ước có một người anh để tặng mình xe đạp., Ước có một chiếc xe đạp đẹp., Ước được đi một vòng trên chiếc xe đạp đẹp., Tất cả đều sai.], [Nhân vật ""tôi"" được người anh tặng cho một chiếc xe đạp rất đẹp nhân dịp sinh nhật của mình., Cậu bé quyết tâm trở thành một người anh có khả năng cho em mình những gì cần thiết., Cậu bé có một người em tàn tật.., Tất cả đều sai.], [Tôi, anh, em., Tớ, anh, em., Tôi, bạn, tớ, anh, em., Bạn, tôi.], [Cậu là một người anh rất yêu thương em trai của mình., Cậu bé là người ích kỷ, chỉ biết lo cho mình., Cậu bé thích đi xe đạp., Tất cả đều sai.], [Ước trở thành người anh biết mua xe đạp tặng em., Ước trở thành người anh yêu thương quan tâm đến 

The dataset contains context, question, and options. Each context could contain multiple training samples

In [ ]:
def explode(df):
    df = df.explode('questions').reset_index(drop=True)
    df['options'] = df['options'].explode().reset_index(drop=True)
    df['answers'] = df['answers'].explode().reset_index(drop=True)
    return df.to_dict()

flattened_data = dataset.with_format('pandas').map(lambda df: explode(df))

# 2. Load model

In [7]:
from transformers import DebertaV2Model
from transformers import AutoTokenizer
from transformers import DebertaV2ForMultipleChoice

model_path = 'Fsoft-AIC/videberta-base'

# Load the model
model = DebertaV2ForMultipleChoice.from_pretrained(model_path)

# Print the model structure
print(model)

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at Fsoft-AIC/videberta-base and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForMultipleChoice(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
 

In [9]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [1, 15942, 6, 7113, 19800, 415, 98427, 73, 2, 22037, 7113, 415, 98427, 5, 88917, 15578, 5, 156, 128, 7, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}